In [124]:
import openpyxl as xl
from decimal import Decimal, getcontext

class Result:
    getcontext().prec = 9
    target_cols = [4,11,12,22]

    def __init__(self, n_of_digits):
        '''self.n_of_digits - кол-во знаков порсле запятой'''
        self.n_of_digits = n_of_digits 
        self.result = []
        
    def load_prepare_make(self, link):
        '''Загружает Excel-таблицу, обрабатывает её, полноценно создает атрибут result'''
        wb = xl.load_workbook(f'таблицы/{link}.xlsx', data_only=True)
        sheet = wb.active
        self.__total_gos_price = Decimal(sheet.cell(row=6,column=7).value)
        for i in range(24,sheet.max_row):
            fin_row = []
            for j in Result.target_cols:
                if j == 4:
                    fin_row.append(str(Result.first_col_creator(sheet.cell(row=i,column=j).value)))
                else:
                    fin_row.append(str(sheet.cell(row=i,column=j).value))
            self.result.append(fin_row)
        for i in self.result:
            i.append(Decimal(i[1]) * Decimal(i[2]))
        for i in self.result:
            i.append(Decimal(i[3]) / Decimal(i[2]) - 1)   

    #################################################################

    @staticmethod
    def first_col_creator(txt):
        '''Метод формирует первый элемент каждой строки списка .result'''
        txt = txt.lower().split()
        ml_num = ['мл', '№']
        first_col = []
        for i in txt:
            for j in ml_num:
                if j in i:
                    first_col.append(i.replace(j,''))
        first_col = [int(i) for i in first_col]
        res = 1
        for i in first_col:
            res *= i
        return res

    #################################################################

    def defining_consts(self):
        '''Определение общей суммы и суммы строки'''
        self.our_total = sum([i[4] for i in self.result])
        self.rows_sum = [i[4] for i in self.result]
        self.difference = self.__total_gos_price - self.our_total

    def update(self):
        '''Обновляет значения суммы строки и процента до предела реестра'''
        for i in range(len(self.result)): #сумма строки
            self.result[i][4] = Decimal(self.result[i][1]) * Decimal(self.result[i][2])
            self.result[i][5] = (Decimal(self.result[i][3]) / Decimal(self.result[i][2])) - 1
        self.defining_consts()

    #################################################################

    def count_up_to_n_decimal_places(self,index):
        '''Подбивает строку под n-знаков'''
        price_of_ml = Decimal(self.result[index][2]) / Decimal(self.result[index][0])
        price_of_ml = str(price_of_ml).split('.')[0] + '.' + str(price_of_ml).split('.')[1][:self.n_of_digits]
        new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        while new_full_price != Decimal(str(new_full_price).split('.')[0] + '.' + str(new_full_price).split('.')[1][:2]) and Decimal(price_of_ml) > 0:
            price_of_ml = str(Decimal(price_of_ml)-Decimal('0.01'))
            new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        self.result[index][2] = str(new_full_price)
        print(new_full_price)


        # price_of_ml = Decimal(self.result[index][2]) / (Decimal(self.result[index][0]))
        # new_price = round(price_of_ml, self.n_of_digits) * Decimal(self.result[index][0])
        # if Decimal(new_price) != Decimal(round(new_price, 2)):
        #     new_price = ((round(price_of_ml, self.n_of_digits) - Decimal(1 / 10 ** (self.n_of_digits))) * Decimal(self.result[index][0]))

        # new_row_price = Decimal(new_price) * Decimal(self.result[index][1])

        # if (new_row_price < self.result[index][4]+self.difference) and (Decimal(new_price) <= Decimal(self.result[index][3])):
        #     self.result[index][2] = str(new_price)        
        # else:
        #     self.result[index][2] = str(((round(price_of_ml,self.n_of_digits) - Decimal(1 / 10 ** (self.n_of_digits))) * Decimal(self.result[index][0])))
        # self.update()
    
    # def count_up_to_n_decimal_places(self,index):
    #     '''ы'''
    #     price_of_ml = Decimal(self.result[index][2]) / (Decimal(self.result[index][0]))
    #     new_price = Decimal(round(price_of_ml, self.n_of_digits)) * Decimal(self.result[index][0])
    #     while Decimal(str(new_price)) != Decimal(str(round(Decimal(new_price), 2))):
    #         new_price = Decimal((round(price_of_ml, self.n_of_digits) - Decimal(1 / 10 ** (self.n_of_digits))) * Decimal(self.result[index][0]))
            
    #     # new_price = round(price_of_ml, self.n_of_digits) * Decimal(self.result[index][0])
    #     # #оберни циклом вайл чтобы много знаков считалось нормально(пока новая цена не примет два знака после запятой)
    #     # if new_price != round(new_price, 2):
    #     #     new_price = str(((round(price_of_ml, self.n_of_digits) - Decimal(1 / 10 ** (self.n_of_digits))) * Decimal(self.result[index][0])))

    #     new_row_price = Decimal(new_price) * Decimal(self.result[index][1])

    #     if (new_row_price < self.result[index][4]+self.difference) and (Decimal(new_price) <= Decimal(self.result[index][3])):
    #         self.result[index][2] = str(new_price)        
    #     else:
    #         self.result[index][2] = str(((round(price_of_ml,self.n_of_digits) - Decimal(1 / 10 ** (self.n_of_digits))) * Decimal(self.result[index][0])))
    #     self.update()

    def divide_to_2_rows(self, index):
        index -= 1
        summ, amount1, NxV = Decimal(self.result[index][4]), self.result[index][1], self.result[index][0]
        price_ml_1 = Decimal(input(f'Цена за мл: {summ / Decimal(amount1) / Decimal(NxV)}\nЦена за мл 1: '))
        price_ml_2 = Decimal(input(f'Цена за мл: {summ / Decimal(amount1) / Decimal(NxV)}\nЦена за мл 2: '))
        lst=[]
        # for i in range(1, int(amount1) + 1):
        #     amount2 = int(amount1) - i
        #     lst[i] = 
        lst2 = [(summ - ((Decimal(amount1) * Decimal(NxV) * Decimal(price_ml_1)) + (Decimal(int(amount1) - i) * Decimal(NxV) * Decimal(price_ml_2)))) for i in range(1,int(amount1))]
        min_diff, min_index = min(lst2), lst.index(min(lst2))
        print(f'Остаток: {min_diff} | Кол-во упаковок по цене {price_ml_1} за мл - {min_index} | Кол-во упаковок по цене {price_ml_2} за мл - {self.result[index][1] - min_index}')
    #################################################################
    def price_price_output(self):
        for j, i in enumerate(self.result, start=1):
            price, const = Decimal(i[2]), Decimal(i[0])
            print(f'{j}) {price}---{price / const}')

    def final_print(self):
        '''Метод финальной печати .result'''
        self.update()
        for i in self.result:
            print(str(i))
        print('Цена госзакупки: ' + str(self.__total_gos_price))
        print('Цена контракта:  ' + str(self.our_total))
        print('Разница:         ' + str(self.difference))


In [102]:
price = 0.0324675
price = str(price).split('.')[0]+'.'+str(price).split('.')[1][:n_of_digits]
price

'0.0324'

In [126]:
res=Result(5)
res.load_prepare_make('0721')
res.update()
for i in range(len(res.result)):
    res.count_up_to_n_decimal_places(i)
res.update()
res.final_print()

336.6000
556.30000
627.84000
573.28000
731.28000
1051.20000
803.52000
730.10000
-8.25600
['6000', '700', '336.6000', '336.65', Decimal('235620.000'), Decimal('0.00014854')]
['10000', '422', '556.30000', '556.47', Decimal('234758.600'), Decimal('0.00030559')]
['6000', '350', '627.84000', '778.56', Decimal('219744.000'), Decimal('0.24006116')]
['8000', '200', '573.28000', '573.41', Decimal('114656.000'), Decimal('0.00022677')]
['8000', '625', '731.28000', '1115.97', Decimal('457050.000'), Decimal('0.52605021')]
['8000', '313', '1051.20000', '1239.69', Decimal('329025.600'), Decimal('0.17930936')]
['6400', '390', '803.52000', '803.59', Decimal('313372.800'), Decimal('0.00008712')]
['10000', '2500', '730.10000', '918.49', Decimal('1825250.00'), Decimal('0.25803315')]
['6400', '3907', '-8.25600', '626.35', Decimal('-32256.1920'), Decimal('-76.8660368')]
Цена госзакупки: 5698116.25
Цена контракта:  3697220.81
Разница:         2000895.44


In [61]:
res.result[7][2] = '739.75'

In [ ]:
index = 8
price = round(Decimal(res.result[index][2]), 2)
const = Decimal(res.result[index][0])
for i in range(50):
    print(f'{price}---{price / const}')
    price -= Decimal('0.01')

res.update()
# res.final_print()

In [ ]:
res.price_price_output()

In [ ]:
index=0

price_of_ml = Decimal(self.result[index][2]) / Decimal(self.result[index][0])
        price_of_ml = str(price_of_ml).split('.')[0] + '.' + str(price_of_ml).split('.')[1][:self.n_of_digits]
        new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        while new_full_price != Decimal(str(new_full_price).split('.')[0] + '.' + str(new_full_price).split('.')[1][:2])
            price_of_ml = str(Decimal(price_of_ml)-Decimal('0.01'))
            new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        print(new_full_price)